## Basic setup and directory structure

In [ ]:
import os

BASE_DIR = '/home/rithomas'
# folder containing MIDI and/or MusicXML files. can have child folders.
RAW_DATA = os.path.join(BASE_DIR, 'data', 'clean_midi')
# TFRecord file that will contain NoteSequence protocol buffers.
SEQUENCES_TFRECORD = os.path.join(BASE_DIR, 'cache', 'melodyRNN', 'data', 'notesequences.tfrecord')
# Dataset of Sequence Examples for the MelodyRNN Model
MELODY_RNN_DATASET = os.path.join(BASE_DIR, 'cache', 'magenta')
# Model config <one of 'basic_rnn', 'mono_rnn', lookback_rnn', or 'attention_rnn'>
MELODY_RNN_CONFIG = 'attention_rnn'
# Model directory for storing weights and tensorboard stuff
MELODY_RNN_DIR = os.path.join(BASE_DIR, 'cache', 'melodyRNN', 'model')
# Training sequence examples
TRAINING_EXAMPLES = os.path.join(MELODY_RNN_DATASET, 'training_melodies.tfrecord')
# Dir for testing outputs
OUTPUTS_PATH = os.path.join(MELODY_RNN_DATASET, 'outputs')
# Final bundled model
BUNDLE_DIR = os.path.join(MELODY_RNN_DIR, MELODY_RNN_CONFIG + '.mag')

if not os.path.exists(SEQUENCES_TFRECORD):
    !python3 "magenta/scripts/convert_dir_to_note_sequences.py" \
      --input_dir=$RAW_DATA \
      --output_file=$SEQUENCES_TFRECORD \
      --recursive

## Create MelodyRNN Dataset

In [ ]:
!python3 "magenta/models/melody_rnn/melody_rnn_create_dataset.py" \
    --config=$MELODY_RNN_CONFIG \
    --input=$SEQUENCES_TFRECORD \
    --output_dir=$MELODY_RNN_DATASET \
    --eval_ratio=0.10

## Train MelodyRNN Model on the Dataset

In [ ]:
!python3 "magenta/models/melody_rnn/melody_rnn_train.py" \
    --config=$MELODY_RNN_CONFIG \
    --run_dir=$MELODY_RNN_DIR \
    --sequence_example_file=$TRAINING_EXAMPLES \
    --hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
    --num_training_steps=20000

## Generate some outputs for testing

In [ ]:
!python3 "magenta/models/melody_rnn/melody_rnn_generate.py" \
    --config=$MELODY_RNN_CONFIG \
    --run_dir=$MELODY_RNN_DIR \
    --output_dir=$OUTPUTS_PATH \
    --num_outputs=10 \
    --num_steps=128 \
    --hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
    --primer_melody="[60]"

## Save model if you like it!

In [ ]:
!python3 "magenta/models/melody_rnn/melody_rnn_generate.py" \
    --config=$MELODY_RNN_CONFIG \
    --run_dir=$MELODY_RNN_DIR \
    --hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
    --bundle_file=$BUNDLE_DIR \
    --save_generator_bundle